In [ ]:
import pandas as pd
import numpy as np
import ppscore as pps
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTENC
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

In [ ]:
import lightgbm

https://core.ac.uk/download/pdf/55631291.pdf

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to assess if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]

# Input variables:
### bank client data:
1 - **age** (**numeric**)<br>
2 - **job** : type of job (**categorical**: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - **marital** : marital status (**categorical**: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - **education** (**categorical**: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>
5 - **default**: has credit in default? (**categorical**: 'no','yes','unknown')<br>
6 - **housing**: has housing loan? (**categorical**: 'no','yes','unknown')<br>
7 - **loan**: has personal loan? (**categorical**: 'no','yes','unknown')<br>
### related with the last contact of the current campaign:
8 - **contact**: contact communication type (**categorical**: 'cellular','telephone')<br>
9 - **month**: last contact month of year (**categorical**: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
10 - **day_of_week**: last contact day of the week (**categorical**: 'mon','tue','wed','thu','fri')<br>
11 - **duration**: last contact duration, in seconds (**numeric**). **DATA LEAKAGE** Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br>
### other attributes:
12 - **campaign**: number of contacts performed during this campaign and for this client (**numeric**, includes last contact)<br>
13 - **pdays**: number of days that passed by after the client was last contacted from a previous campaign (**numeric**; 999 means client was not previously contacted)<br>
14 - **previous**: number of contacts performed before this campaign and for this client (**numeric**)<br>
15 - **poutcome**: outcome of the previous marketing campaign (**categorical**: 'failure','nonexistent','success')<br>
### social and economic context attributes
16 - **emp.var.rate**: employment variation rate - quarterly indicator (**numeric**)<br>
17 - **cons.price.idx**: consumer price index - monthly indicator (**numeric**)<br>
18 - **cons.conf.idx**: consumer confidence index - monthly indicator (**numeric**)<br>
19 - **euribor3m**: euribor 3 month rate - daily indicator (**numeric**)<br>
20 - **nr.employed**: number of employees - quarterly indicator (**numeric**)<br>

# Target variable:
21 - **y** - has the client subscribed a term deposit? (binary: 'yes','no')

In [ ]:

# look at these person attributes, these can't be changed
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']

# attributes for the economy
['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

# attributes for last campaign
['previous', 'poutcome']

# attributes with current campaign
['contact', 'month', 'day_of_week', 'campaign', 'pdays']


In [ ]:
train = pd.read_csv('train.csv')

# drop duration column due to its 'unrealistic' effect on the target variable
train = train.drop(columns='duration')
train.head()

# look at economic factors

### scatter plots are useless due to the fact that the dots just sit on top of each other

In [ ]:
# econ = train.loc[:, ['y', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']]
# econ.head()

In [ ]:
# f, ((ax1, ax2, ax3), (ax4, ax5, _)) = plt.subplots(nrows=2, ncols=3, figsize=(30,10))
# sns.boxenplot(x='emp_var_rate', y='y', data=econ, ax=ax1)
# sns.boxenplot(x='cons_price_idx', y='y', data=econ, ax=ax2)
# sns.boxenplot(x='cons_conf_idx', y='y', data=econ, ax=ax3)
# sns.boxenplot(x='euribor3m', y='y', data=econ, ax=ax4)
# sns.boxenplot(x='nr_employed', y='y', data=econ, ax=ax5);

In [ ]:
# f, ((ax1, ax2, ax3), (ax4, ax5, _)) = plt.subplots(nrows=2, ncols=3, figsize=(30,10))
# sns.histplot(x='emp_var_rate', hue='y', data=econ, ax=ax1)
# sns.histplot(x='cons_price_idx', hue='y', data=econ, ax=ax2)
# sns.histplot(x='cons_conf_idx', hue='y', data=econ, ax=ax3)
# sns.histplot(x='euribor3m', hue='y', data=econ, ax=ax4)
# sns.histplot(x='nr_employed', hue='y', data=econ, ax=ax5);

# Balance the Data using combo of Over (with SMOTE) and Under sampling methods

In [ ]:
train.y.value_counts()

In [ ]:
# under = RandomUnderSampler(sampling_strategy=.2, random_state=36)
# X_resampled, y_resampled = under.fit_resample(train.drop(columns='y'), train.y)
# X_resampled['y'] = y_resampled

In [ ]:
# ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
cat_locations = [1, 2, 3, 4, 5, 6, 7, 8, 9, 13]
over = SMOTENC(categorical_features=cat_locations, random_state=36)

In [ ]:
X_resampled, y_resampled = over.fit_resample(train.drop(columns='y'), train.y)
X_resampled['y'] = y_resampled

In [ ]:
X_resampled.y.value_counts()

In [ ]:
train = X_resampled.copy()

In [ ]:
def percent_of_target(col):
    x = train.groupby([col, 'y'])[['age']].count().unstack()
    x.columns = ['no', 'yes']
    x/=x.sum()

    no = x.loc[:, ['no']]
    no.columns = ['value']
    no['target'] = 'no'

    yes = x.loc[:, ['yes']]
    yes.columns = ['value']
    yes['target'] = 'yes'

    return pd.concat([no, yes]).reset_index()


def percent_of_category(col):
    x = train.groupby(['y', col])[['age']].count().unstack()
    x/=x.sum()
    x = x.T
    x.columns = ['no', 'yes']

    no = x.loc[:, ['no']]
    no.columns = ['value']
    no['target'] = 'no'

    yes = x.loc[:, ['yes']]
    yes.columns = ['value']
    yes['target'] = 'yes'

    return pd.concat([no, yes]).reset_index().drop(columns='level_0')

In [ ]:
f, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(nrows=5, ncols=2, figsize=(30,20))

# graph_func = percent_of_target
graph_func = percent_of_category

column = 'job'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax1)

column = 'marital'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax2)

column = 'education'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax3)

column = 'default'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax4)

column = 'housing'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax5)

column = 'loan'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax6)

column = 'contact'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax7)

column = 'month'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax8)

column = 'day_of_week'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax9)

column = 'poutcome'
d = graph_func(column)
sns.barplot(data=d, x=column, y='value', hue='target', ax=ax10);

In [ ]:
f, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8 , ax9)) = plt.subplots(nrows=3, ncols=3, figsize=(30,13))
sns.histplot(data=train, x='age', hue='y', kde=True, ax=ax1)
sns.histplot(data=train, x='campaign', hue='y', kde=True, ax=ax2)
sns.histplot(data=train, x='pdays', hue='y', kde=True, ax=ax3)
sns.histplot(data=train, x='previous', hue='y', kde=True, ax=ax4)
sns.histplot(data=train, x='emp_var_rate', hue='y', kde=True, ax=ax5)
sns.histplot(data=train, x='cons_price_idx', hue='y', kde=True, ax=ax6)
sns.histplot(data=train, x='cons_conf_idx', hue='y', kde=True, ax=ax7)
sns.histplot(data=train, x='euribor3m', hue='y', kde=True, ax=ax8)
sns.histplot(data=train, x='nr_employed', hue='y', kde=True, ax=ax9);

# replace category strings with integer values

In [ ]:
train.y = train.y.astype('category')

train.job = train.job.astype('category')
train.marital = train.marital.astype('category')
train.education = train.education.astype('category')
train.default = train.default.astype('category')
train.housing = train.housing.astype('category')
train.loan = train.loan.astype('category')
train.contact = train.contact.astype('category')
train.month = train.month.astype('category')
train.day_of_week = train.day_of_week.astype('category')
train.poutcome = train.poutcome.astype('category')

X = train.drop(columns='y')

# X.job = X.job.cat.codes
# X.marital = X.marital.cat.codes
# X.education = X.education.cat.codes
# X.default = X.default.cat.codes
# X.housing = X.housing.cat.codes
# X.loan = X.loan.cat.codes
# X.contact = X.contact.cat.codes
# X.month = X.month.cat.codes
# X.day_of_week = X.day_of_week.cat.codes
# X.poutcome = X.poutcome.cat.codes

Y = train.y.cat.codes

In [ ]:
Y

In [ ]:
X

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (roc_auc_score,
                             roc_curve,
                             f1_score,
                             recall_score,
                             precision_score,
                             precision_recall_curve,
                             classification_report)




In [ ]:
t = pd.get_dummies(X)
t

In [ ]:
t.columns

In [ ]:
X_new = X.loc[:, ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']].copy()
X_new = pd.get_dummies(X_new)
X_new.shape

In [ ]:
x, x_test, y, y_test = train_test_split(X_new, Y, test_size=0.2, random_state=36, stratify=Y)

In [ ]:
clf = LogisticRegression(random_state=36, max_iter=400).fit(x, y)
clf.score(x, y)

In [ ]:
clf.score(x_test, y_test)

In [ ]:
clf.predict_proba(x_test)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, precision_score, precision_recall_curve, classification_report

In [ ]:
# Create the LightGBM data containers

cat_locations = [1, 2, 3, 4, 5, 6, 7, 8, 9, 13]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=cat_locations)
test_data = lightgbm.Dataset(x_test, label=y_test)

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'boosting': 'gbdt',
#     'num_leaves': 31,
    'num_leaves': 2,
#     'feature_fraction': 0.5,
#     'bagging_fraction': 0.5,
#     'bagging_freq': 20,
#     'learning_rate': 0.05,
#     'verbose': 0
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       categorical_feature=cat_locations,
                       num_boost_round=3000,
                       early_stopping_rounds=100)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, precision_score, precision_recall_curve, classification_report

In [ ]:
y_pred = model.predict(x)
y_pred = pd.Series(y_pred)

roc_auc_score(y, y_pred)
# fpr, tpr, thresholds = roc_curve(y, y_pred)
# sns.scatterplot(x=fpr, y=tpr);

In [ ]:
y_pred = model.predict(x_test)
y_pred = pd.Series(y_pred)

roc_auc_score(y_test, y_pred)
# fpr, tpr, thresholds = roc_curve(y_test, y_pred)
# sns.scatterplot(x=fpr, y=tpr);

In [ ]:
for i in np.arange(.1, 1, .1):
    print(f'{i}------------------------------------------------------\n',
          classification_report(y_test, y_pred.apply(lambda x: 1 if x >= i else 0)),
          '-----------------------------------------------------------\n\n\n\n')

# take a look at hold out data

In [ ]:
test = pd.read_csv('test.csv')

# drop duration column due to its 'unrealistic' effect on the target variable
test = test.drop(columns='duration')
test.head()

test_cat = test.copy()
for cat_col in train.select_dtypes('category').columns:
    l = train[cat_col].cat.categories.to_list()

    test_cat[cat_col] = test_cat[cat_col].apply(lambda x: l.index(x))

In [ ]:
X_holdout = test_cat.drop(columns='y')
y_holdout = test_cat.y

X_display = test.drop(columns='y')
y_display = test.y

In [ ]:
y_pred = model.predict(X_holdout)
y_pred = pd.Series(y_pred)
y_pred

In [ ]:
# comparing = pd.DataFrame(y_holdout)
# comparing['y_pred'] = y_pred
# comparing['y_thresholded'] = comparing.y_pred.apply(lambda x: 0 if x < .5 else 1)
# comparing 

In [ ]:
# comparing.groupby(['y', 'y_thresholded']).count()

In [ ]:
# comparing.y.value_counts()

In [ ]:
# tpr = 546/1392
# fpr = 683/10964
# fnr = 846/1392
# tnr = 10281/10964

# OR = (tpr/fpr)/(fnr/tnr)
# np.log(OR), np.log10(OR)

In [ ]:
roc_auc_score(y_holdout, y_pred)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_holdout, y_pred)
sns.scatterplot(x=fpr, y=tpr);

In [ ]:
import shap

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_holdout)

# A No example

In [ ]:
X_display.iloc[0,:]

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], X_display.iloc[0,:])

# A Yes example

In [ ]:
X_display.iloc[-1,:]

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][-1,:], X_display.iloc[-1,:])

In [ ]:
shap.force_plot(explainer.expected_value[1], np.vstack((shap_values[1][:500,:],shap_values[1][-500:,:])), pd.concat([X_display.iloc[:500,:],X_display.iloc[-500:,:]]))

In [ ]:
shap.force_plot(explainer.expected_value[1], np.vstack((shap_values[1][:500,:],shap_values[1][-500:,:])), pd.concat([X_display.iloc[:500,:],X_display.iloc[-500:,:]]))

In [ ]:
shap.force_plot(explainer.expected_value[1], np.vstack((shap_values[1][:500,:],shap_values[1][-500:,:])), pd.concat([X_display.iloc[:500,:],X_display.iloc[-500:,:]]))

In [ ]:
shap.summary_plot(shap_values[1], X_holdout, plot_type='violin')

In [ ]:
shap.summary_plot(shap_values[1], X_holdout)

In [ ]:
# shap.summary_plot(shap_values[1], X_holdout, plot_type='violin')
# shap.summary_plot(shap_values[1], X_holdout)
shap.summary_plot(shap_values, X_holdout)

In [ ]:
# from sklearn.metrics import roc_auc_score, roc_curve
# from sklearn.model_selection import train_test_split
# import neptunecontrib.monitoring.skopt as sk_utils
# import lightgbm as lgb
# import pandas as pd
# import neptune
# import skopt
# import sys
# import os

# SEARCH_PARAMS = {'learning_rate': 0.4,
#                 'max_depth': 15,
#                 'num_leaves': 32,
#                 'feature_fraction': 0.8,
#                 'subsample': 0.2}

# FIXED_PARAMS={'objective': 'binary',
#              'metric': 'auc',
#              'is_unbalance':True,
#              'bagging_freq':5,
#              'boosting':'dart',
#              'num_boost_round':300,
#              'early_stopping_rounds':30}

# def train_evaluate(search_params):
#    # you can download the dataset from this link(https://www.kaggle.com/c/santander-customer-transaction-prediction/data)
#    # import Dataset to play with it
#    data= pd.read_csv("sample_train.csv")
#    X = data.drop(['ID_code', 'target'], axis=1)
#    y = data['target']
#    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1234)
#    train_data = lgb.Dataset(X_train, label=y_train)
#    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

#    params = {'metric':FIXED_PARAMS['metric'],
#              'objective':FIXED_PARAMS['objective'],
#              **search_params}

#    model = lgb.train(params, train_data,                     
#                      valid_sets=[valid_data],
#                      num_boost_round=FIXED_PARAMS['num_boost_round'],
#                      early_stopping_rounds=FIXED_PARAMS['early_stopping_rounds'],
#                      valid_names=['valid'])
#    score = model.best_score['valid']['auc']
#    return score

In [ ]:
# neptune.init('mjbahmani/LightGBM-hyperparameters')
# neptune.create_experiment('lgb-tuning_final', upload_source_files=['*.*'],
#                               tags=['lgb-tuning', 'dart'],params=SEARCH_PARAMS)

# SPACE = [
#    skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),
#    skopt.space.Integer(1, 30, name='max_depth'),
#    skopt.space.Integer(10, 200, name='num_leaves'),
#    skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),
#    skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')
# ]
# @skopt.utils.use_named_args(SPACE)
# def objective(**params):
#    return -1.0 * train_evaluate(params)

# monitor = sk_utils.NeptuneMonitor()
# results = skopt.forest_minimize(objective, SPACE, 
#                                 n_calls=100, n_random_starts=10, 
#                                 callback=[monitor])
# sk_utils.log_results(results)

# neptune.stop()

In [ ]:
predictors_df = pps.predictors(dummy_train, y="y")
predictors_df

In [ ]:
f, ax = plt.subplots(figsize=(30,5))
sns.barplot(data=predictors_df.head(12), x="x", y="ppscore", ax=ax);

In [ ]:
f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(ncols=3, nrows=2, figsize=(30,10))
sns.boxplot(x='age', y='y', data=train, ax=ax1)
sns.boxplot(x='campaign', y='y', data=train, ax=ax2)
sns.boxplot(x='pdays', y='y', data=train, ax=ax3)
sns.boxplot(x='previous', y='y', data=train, ax=ax4)
sns.boxplot(x='emp_var_rate', y='y', data=train, ax=ax5)
sns.boxplot(x='cons_price_idx', y='y', data=train, ax=ax6);

In [ ]:
train.head().T

In [ ]:
train.loc[train.y=='no',:].hist(bins=10, layout=(2,5), figsize=(30,10), color='g');
train.loc[train.y=='yes',:].hist(bins=10, layout=(2,5), figsize=(30,10), color='b');

In [ ]:
train.select_dtypes('object').head()

In [ ]:
pd.get_dummies(train.select_dtypes('object')).head()

In [ ]:
pd.get_dummies(train).head()

In [ ]:
over = SMOTE(sampling_strategy=0.1, k_neighbors=k)
under = RandomUnderSampler(sampling_strategy=0.5)

In [ ]:
SMOTE?

In [ ]:
sm = SMOTE(random_state=36, ratio = 1.0)
# x_train_res, y_train_res = sm.fit_sample(x_train, y_train)